# simple transformer

In [18]:
import pandas as pd
df  = pd.read_csv('/Users/raunakpandey/Documents/programming/projects/augmentation/augmented_data/final_dataset/Final_Dataset.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3804 entries, 0 to 3803
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   premises  3804 non-null   object
 1   Text      3804 non-null   object
 2   label     3804 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 89.3+ KB


In [20]:
train_df = pd.DataFrame(columns=['premises', 'Text', 'label'])
test_df = pd.DataFrame(columns=['premises', 'Text', 'label'])

In [21]:
j =0
for i in range(len(df)):
    j = i%317
    df2 = pd.DataFrame({'premises': [df.iloc[i,0]], 'Text': [df.iloc[i,1]], 'label': [df.iloc[i,2]]})
    if j>=0 and j<250:
        train_df = pd.concat([train_df, df2], ignore_index=True)
    else:
        test_df = pd.concat([test_df, df2], ignore_index=True)

In [34]:
train_df.iloc[1750]

premises    The cat is currently slumbering on the window ...
Text                   The dog is sleeping on the windowsill.
label                                                       0
Name: 1750, dtype: object

In [22]:
# Shuffle the DataFrame
train_df = train_df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Reset the index to make it continuous
train_df = train_df.reset_index(drop=True)

In [23]:
# Shuffle the DataFrame
test_df = test_df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Reset the index to make it continuous
test_df = test_df.reset_index(drop=True)

In [24]:
test_df.head(5)

,premises,Text,label
0,A startup in the renewable energy industry is ...,Innovative solar concentrator technology.,1
1,Engineers are working on a high-efficiency sol...,There is no team of engineers working on a cos...,0
2,Engineers are in the process of creating a dro...,Unmanned aerial vehicles for wildlife protection.,1
3,Scientists are exploring the possibility of us...,A tetrathlon of researchers is studying the po...,0
4,Researchers are exploring the impact of music ...,The psychological team is not currently invest...,0


In [25]:
eval_df = test_df.copy()

In [26]:
train_df = train_df.rename(columns={'premises': 'text_a', 'Text': 'text_b', 'label': 'labels'}).dropna()
eval_df = eval_df.rename(columns={'premises': 'text_a', 'Text': 'text_b', 'label': 'labels'}).dropna()

In [27]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  804 non-null    object
 1   text_b  804 non-null    object
 2   labels  804 non-null    object
dtypes: object(3)
memory usage: 19.0+ KB


In [46]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'max_seq_length': 512,
    'num_train_epochs': 10,
    'evaluate_during_training_steps': 50,
    'train_batch_size': 16,
    'regression': False,
}

In [29]:
from scipy.stats import pearsonr, spearmanr


def pearson_corr(preds, labels):
    return pearsonr(preds, labels)[0]


def spearman_corr(preds, labels):
    return spearmanr(preds, labels)[0]

In [5]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('roberta','/Users/raunakpandey/Documents/programming/projects/augmentation/model_colab', use_cuda=False)

In [6]:
predictions, raw_outputs = model.predict([["The man is sleeping with his wife.", "The man is resting."]])

100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  9.51it/s]


In [7]:
predictions

array([1])

In [8]:
raw_outputs

array([[-5.59428358,  5.63537788]])

In [15]:
predictions, raw_outputs = model.predict([["animal drinks water.", "the student is studying."]])

100%|██████████| 1/1 [00:02<00:00,  2.73s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00, 11.39it/s]


In [17]:
predictions

array([1])

In [32]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

  0%|          | 2/804 [00:02<19:35,  1.47s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Running Evaluation: 100%|██████████| 101/101 [00:42<00:00,  2.39it/s]


In [33]:
print(result)

{'mcc': 1.0, 'tp': 402, 'tn': 402, 'fp': 0, 'fn': 0, 'auroc': 1.0, 'auprc': 1.0, 'pearson_corr': 1.0, 'spearman_corr': 1.0, 'eval_loss': 1.4659518897268313e-05}


In [35]:
predictions, raw_outputs = model.predict([["The man is sleeping with his wife.", "The man is resting."]])
print(predictions)

100%|██████████| 1/1 [00:02<00:00,  2.82s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  6.95it/s]

[1]


In [36]:
predictions, raw_outputs = model.predict([["The man is sleeping with his wife.", "The man is not sleeping"]])
print(predictions)

100%|██████████| 1/1 [00:02<00:00,  2.73s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00, 12.39it/s]

[0]


In [37]:
predictions, raw_outputs = model.predict([["The man is sleeping with his wife.", "The woman is sleeping."]])
print(predictions)

100%|██████████| 1/1 [00:02<00:00,  2.67s/it]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00, 12.63it/s]

[1]
